# totoの平均当選金額、当選確率から期待値を計算する


## WEBスクレイピングによる当選金額データの取得
* toto公式サイト（ http://sport-kuji.toto-dream.com/dci/I/IPB/IPB01.do?op=initLotResultLsttoto ）からWEBスクレイピングで当選金額データを取得する
* 過去の結果を閲覧できるURLは http://sport-kuji.toto-dream.com/dci/I/IPB/IPB01.do?op=lnkHoldCntLotResultLsttoto&holdCntId={開催回}
* {開催回}の部分の数字を変えれば、各回のURLにアクセスすることができる

### pandasを使用して対象URLからテーブル要素を取得
toto、mini toto-A組、mini toto-B組、totoGOAL3のデータだけを抽出するために、pandasで取得したデータに規則性があるかを調査する

In [1]:
import pandas
from IPython.core.display import display

target_url_sample = 'http://sport-kuji.toto-dream.com/dci/I/IPB/IPB01.do?op=lnkHoldCntLotResultLsttoto&holdCntId=0208'
fetched_dataframes_sample = pandas.io.html.read_html(target_url_sample)

In [2]:
display(fetched_dataframes_sample[3])

,0,1,2
0,第208回,1等,2等
1,当せん金,"625,175円","9,230円"
2,当せん口数,140口,"2,370口"
3,次回への繰越金,0円,0円


In [3]:
display(fetched_dataframes_sample[9])

,0,1
0,第208回,1等
1,当せん金,"1,196円"
2,当せん口数,"15,903口"
3,次回への繰越金,0円


In [4]:
display(fetched_dataframes_sample[15])

,0,1
0,第208回,1等
1,当せん金,"4,298円"
2,当せん口数,"3,536口"
3,次回への繰越金,0円


In [5]:
display(fetched_dataframes_sample[21])

,0,1,2
0,第208回,1等,2等
1,当せん金,"100,798円","3,320円"
2,当せん口数,92口,"1,862口"
3,次回への繰越金,0円,0円


pandasで取得したdataframeの下記要素番号にtoto、mini toto-A組、mini toto-B組、totoGOAL3の当選結果が格納されることがわかる。

|  要素番号 |  格納データ  |
| ---- | ---- |
| 3 | toto |
| 9 | mini toto-A組 |
| 15 | mini toto-B組 |
| 21 | totoGOAL3 |

しかし、毎回この4種類の宝くじが開催されるとは限らない。

totoのない回はダイレクトに要素番号を指定すると期待通りのデータが取得できない。

よって下記の処理を行う。

1. 全てのdataframe要素について、行番号1、列番号0の文字列が"当せん金"かどうかを判定する
1. 行番号1、列番号0が"当せん金"の要素は当選結果のdataframeとみなす
1. 行番号1、列番号0が"当せん金"の要素番号から-2した要素番号の行番号0、列番号0を取得する
1. 取得した文字列に、特定の文字列が含まれているか判定する
    - 取得した文字列に"回 toto　くじ結果"が含まれて入れば処理1で取得した当選結果はtotoとみなす
    - 取得した文字列に"mini toto-A組"が含まれて入れば処理1で取得した当選結果はmini toto-A組とみなす
    - 取得した文字列に"mini toto-B組"が含まれて入れば処理1で取得した当選結果はmini toto-B組とみなす
    - 取得した文字列に"totoGOAL3"が含まれて入れば処理1で取得した当選結果はtotoGOAL3とみなす

In [6]:
def separateLotteryType(df):
    
    DfToto = pandas.DataFrame()
    DfTotoMiniA = pandas.DataFrame()
    DfTotoMiniB = pandas.DataFrame()
    DfTotoGoal3 = pandas.DataFrame()
    
    for index, item in enumerate(df):
        if len(item.isnull()) < 2:
            continue

        if item.iat[1, 0] == '当せん金':
            if df[index - 2].iat[0,0].find('回 toto　くじ結果') != -1:
                DfToto = item
            elif df[index - 2].iat[0,0].find('mini toto-A組') != -1:
                DfTotoMiniA = item
            elif df[index - 2].iat[0,0].find('mini toto-B組') != -1:
                DfTotoMiniB = item
            elif df[index - 2].iat[0,0].find('totoGOAL3') != -1:
                DfTotoGoal3 = item
    
    return DfToto, DfTotoMiniA, DfTotoMiniB, DfTotoGoal3

2006年の第198回から2018年の第1042回までのデータを取得し、separateLotteryType()をつかってtoto、mini toto-A組、mini toto-B組、totoGOAL3の当選結果に分類する。

In [7]:
from tqdm import tqdm

# toto、mini toto-A組、mini toto-B組、totoGOAL3の当選結果を格納する配列
DfArrToto = []
DfArrTotoMiniA = []
DfArrTotoMiniB = []
DfArrTotoGoal3 = []

for lotteryNo in tqdm(range(198,1043)):
    if lotteryNo < 1000:
        number = '0' + str(lotteryNo)
    else:
        number = str(lotteryNo)
    
    target_url = 'http://sport-kuji.toto-dream.com/dci/I/IPB/IPB01.do?op=lnkHoldCntLotResultLsttoto&holdCntId='+ str(number) 

    fetched_dataframes = pandas.io.html.read_html(target_url)
    DfToto, DfTotoMiniA, DfTotoMiniB, DfTotoGoal3 = separateLotteryType(fetched_dataframes)
    
    if DfToto.empty == False:
        DfArrToto.append(DfToto)
        
    if DfTotoMiniA.empty == False:
        DfArrTotoMiniA.append(DfTotoMiniA)
        
    if DfTotoMiniB.empty == False:
        DfArrTotoMiniB.append(DfTotoMiniB)

    if DfTotoGoal3.empty == False:
        DfArrTotoGoal3.append(DfTotoGoal3)
    
#   print(target_url)


100%|██████████| 845/845 [02:33<00:00,  5.33it/s]


## 賞金総合計、開催回数、平均償金額を計算
### totoの賞金総合計、開催回数、平均償金額を計算

In [8]:
totoTotalFirstPrize = 0
totoTotalSecondPrize = 0
totoHeldNumber = len(DfArrToto)

for item in DfArrToto:

    firstPrize = int(item.iat[1,1].replace("円", "").replace(",", ""))
    secondPrize = int(item.iat[1,2].replace("円", "").replace(",", ""))
    
    totoTotalFirstPrize += firstPrize
    totoTotalSecondPrize += secondPrize

totoAverageFirstPrize = totoTotalFirstPrize / totoHeldNumber
totoAverageSecondPrize = totoTotalSecondPrize / totoHeldNumber

print('1等賞金総合計：' + str(totoTotalFirstPrize) + '円')
print('2等賞金総合計：' + str(totoTotalSecondPrize) + '円')
print('開催回数：' + str(totoHeldNumber) + '回')
print('1等平均賞金：' + str(totoAverageFirstPrize) + '円')
print('２等平均賞金：' + str(totoAverageSecondPrize) + '円')


1等賞金総合計：14307567772円
2等賞金総合計：478922741円
開催回数：507回
1等平均賞金：28220054.777120315円
２等平均賞金：944620.7909270216円


### mini toto-A組の賞金総合計、開催回数、平均償金額を計算

In [9]:
totoMiniATotalPrize = 0
totoMiniAHeldNumber = len(DfArrTotoMiniA)

for item in DfArrTotoMiniA:

    firstPrize = int(item.iat[1,1].replace("円", "").replace(",", ""))
    
    totoMiniATotalPrize += firstPrize

totoMiniAAveragePrize = totoMiniATotalPrize / totoMiniAHeldNumber

print('1等賞金総合計：' + str(totoMiniATotalPrize) + '円')
print('開催回数：' + str(totoMiniAHeldNumber) + '回')
print('1等平均賞金：' + str(totoMiniAAveragePrize) + '円')

1等賞金総合計：10353342円
開催回数：727回
1等平均賞金：14241.18569463549円


### mini toto-B組の賞金総合計、開催回数、平均償金額を計算

In [10]:
totoMiniBTotalPrize = 0
totoMiniBHeldNumber = len(DfArrTotoMiniB)

for item in DfArrTotoMiniB:

    firstPrize = int(item.iat[1,1].replace("円", "").replace(",", ""))
    
    totoMiniBTotalPrize += firstPrize
    
totoMiniBAveragePrize = totoMiniBTotalPrize / totoMiniBHeldNumber

print('1等賞金総合計：' + str(totoMiniBTotalPrize) + '円')
print('開催回数：' + str(totoMiniBHeldNumber) + '回')
print('1等平均賞金：' + str(totoMiniBAveragePrize) + '円')

1等賞金総合計：9449796円
開催回数：598回
1等平均賞金：15802.334448160535円


### totoGOAL3の賞金総合計、開催回数、平均償金額を計算

In [11]:
totoGoal3TotalFirstPrize = 0
totoGoal3TotalSecondPrize = 0
totoGoal3HeldNumber = len(DfArrTotoGoal3)

for item in DfArrTotoGoal3:

    firstPrize = int(item.iat[1,1].replace("円", "").replace(",", ""))
    secondPrize = int(item.iat[1,2].replace("円", "").replace(",", ""))
    
    totoGoal3TotalFirstPrize += firstPrize
    totoGoal3TotalSecondPrize += secondPrize

totoGoal3AverageFirstPrize = totoGoal3TotalFirstPrize / totoGoal3HeldNumber
totoGoal3AverageSecondPrize = totoGoal3TotalSecondPrize / totoGoal3HeldNumber

print('1等賞金総合計：' + str(totoGoal3TotalFirstPrize) + '円')
print('2等賞金総合計：' + str(totoGoal3TotalSecondPrize) + '円')
print('開催回数：' + str(totoGoal3HeldNumber) + '回')
print('1等平均賞金：' + str(totoGoal3AverageFirstPrize) + '円')
print('２等平均賞金：' + str(totoGoal3AverageSecondPrize) + '円')

1等賞金総合計：131472906円
2等賞金総合計：3388720円
開催回数：764回
1等平均賞金：172084.9554973822円
２等平均賞金：4435.497382198952円


## 各宝くじの当選確率と期待値を計算

期待値の計算用に、順列(Permutation)と組み合わせ(Combination)の関数を定義する。

In [12]:
import math

def P(n, r):
    return math.factorial(n)//math.factorial(n-r)

def C(n, r):
    return P(n, r)//math.factorial(r)

### totoの当選確率と期待値

条件
- 1回の試合につき勝ち／引き分け／負けのいずれか1つを選択することとする。
- 実際の宝くじでは1回の試合で勝ち／引き分け／負けの複数を指定する買い方ができるが、そのような買い方はしないものとする。
- 運任せで買った場合の当選確率を計算する

1等当選確率は13回全て当たった場合なので下記式で表される。
\begin{equation}
1等当選確率 = \left( \frac{1}{3} \right)^{13}
\end{equation}

2等当選確率は12回当たって1回外れた場合と、11回当たって2回外れた場合の和なので下記式で表される。
\begin{equation}
2等当選確率 = \left( \frac{1}{3} \right)^{12}  \left( \frac{2}{3} \right)^{1}  {}_{13} C_1 + \left( \frac{1}{3} \right)^{11} \left( \frac{2}{3} \right)^{2} {}_{13} C_2 
\end{equation}

In [13]:
p_firstPrize = math.pow(1/3, 13)
p_secondPrize = math.pow(1/3, 12) * math.pow(2/3, 1) * C(13, 1) + math.pow(1/3, 11) * math.pow(2/3, 2) * C(13, 2) 
print('1等当選確率：' + str(p_firstPrize))
print('2等当選確率：' + str(p_secondPrize))
print('期待値：' + str((totoAverageFirstPrize * p_firstPrize) + (totoAverageSecondPrize * p_secondPrize)) + '円')

1等当選確率：6.272254743863065e-07
2等当選確率：0.0002120022103425716
期待値：217.96203285686366円


### mini toto-A組の当選確率と期待値

条件や確率の計算方法はtotoと同様

In [14]:
p_miniAFirstPrize = math.pow(1/3, 5)
print('1等当選確率：' + str(p_miniAFirstPrize))
print('期待値：' + str(totoMiniAAveragePrize * p_miniAFirstPrize) + '円')

1等当選確率：0.004115226337448558
期待値：58.605702447059606円


### mini toto-B組の当選確率と期待値

条件や確率の計算方法はtotoと同様

In [15]:
p_miniBFirstPrize = math.pow(1/3, 5)
print('1等当選確率：' + str(p_miniBFirstPrize))
print('期待値：' + str(totoMiniBAveragePrize * p_miniBFirstPrize) + '円')

1等当選確率：0.004115226337448558
期待値：65.03018291424087円


### totoGOAL3の当選確率と期待値

条件や確率の計算方法はtotoと同様

In [16]:
p_Goal3FirstPrize = math.pow(1/4, 6)
p_Goal3SecondPrize = math.pow(1/4, 5) * math.pow(3/4, 1) * C(6, 1) 
print('1等当選確率：' + str(p_Goal3FirstPrize))
print('2等当選確率：' + str(p_Goal3SecondPrize))
print('期待値：' + str((totoGoal3AverageFirstPrize * p_Goal3FirstPrize) + (totoGoal3AverageSecondPrize * p_Goal3SecondPrize)) + '円')

1等当選確率：0.000244140625
2等当選確率：0.00439453125
期待値：61.504860443594566円


# 結論
- 期待値はtoto >>>>>> mini toto-B > totoGOAL3 > mini toto-A
- totoの期待値が約218円となっているが、計算間違えた？